In [84]:
#1. given four coordinates and a rotation angle, get XatTopEdge and YatLeftEdge.
chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')
tl, 
#2. add x to top edge and y at left edge to get the top-left chiroptera point.

#3. calculate the reset between the chiroptera and atl, and add it to atl. 

#4. add the rotated offset to the atl.

#5. run correlation!

In [109]:
import pandas as pd
import numpy as np
import math

#Because the image matcher matches ATL and chiroptera images from the top left,
#it is necessary to geolocate the original top left corner of the Chiroptera.
#To do this, we unrotate the original Chiroptera LiDAR (which is at an angle), gets (xmin, ymax), and re-rotate it.

def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

(570762.9061543589, 9497299.94695935)


In [148]:
def get_chirop_tl_coord(i, ANGLE):
    b1 = i * 4
    b2 = b1 + 4
    df = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')
    ANGLE = np.deg2rad(ANGLE)
    center = ((min(df['x']) + max(df['x'])) / 2, (min(df['y']) + max(df['y'])) / 2)

    for index, row in df.iterrows():
        (x,y) = rotate(center, (row['x'], row['y']), ANGLE)
        row['x'], row['y'] = (x,y)

    (rx, ry) = (min(df['x']), max(df['y']))
    (urx, ury) = rotate(center, (rx, ry), -ANGLE)
    #df.to_csv('rot_debug_chirop_df.csv', index=False)
    return (urx, ury)

In [149]:
import geopandas as gpd

atl_raster_ymins = []
for i in range(0, 8):
    b1 = i * 4
    b2 = b1 + 4
    gdf = gpd.read_file(f'../make_image/atl 1C/atl json/atl_{b1}_{b2}.json')
    xmin, ymin, xmax, ymax = gdf.total_bounds
    atl_raster_ymins.append(ymin)

In [160]:
import math

import pandas as pd
import geopandas as gpd

chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')

min_loc_val = [(33, 398),
 (46, 231),
 (66, 75),
 (85, 226),
 (146, 588),
 (205, 426),
 (83, 160),
 (305, 850)]

step = -0.18
ROT_C = np.arange(17.88, 17.88 + 8 * step, step)

def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def get_offset(i, min_loc_val, ANGLE):
    atl_df = pd.read_csv('unshifted_gt3r_full.csv')
    b1 = i * 4
    b2 = b1 + 4
    chirop_df = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')
    x_min = min(chirop_df['x'])
    x_max = max(chirop_df['x'])
    y_min = min(chirop_df['y'])
    y_max = max(chirop_df['y'])
    #getting the correct cutoffs here is critical for the shift, because we base our shift on the top-left point.
    K = 133
    #These bounds come directly from the original georeferenced ATL.
    ATL_max_y = atl_raster_ymins[i] + 4000 + K
    ATL_min_y = ATL_max_y - 2000

    atl_df = atl_df[(atl_df['y'] >= ATL_min_y) & (atl_df['y'] <= ATL_max_y)]
    TL_POINT = get_chirop_tl_coord(i, ANGLE)

    #revert ATL to Chirop TL
    x_max = max(atl_df['x'])
    y_max = max(atl_df['y'])

    offset = (TL_POINT[0] - x_max, TL_POINT[1] - y_max)
    atl_df['x'] = atl_df['x'] + offset[0]
    atl_df['y'] = atl_df['y'] + offset[1]

    #At this point, the ATL has been matched to the top-left corner.
    #SHIFT WITH BEST FIT (from image_matcher)
    #(x,y) (?)
    best_fit = min_loc_val[i]
    best_fit = rotate((0,0), best_fit, np.deg2rad(ANGLE))
    print(best_fit)

    #the calculated y shift is in the negative direction, so we reverse it here.
    atl_df['x'] = atl_df['x'] + best_fit[0]
    atl_df['y'] = atl_df['y'] - best_fit[1]
    atl_df.to_csv(f'debug_shifted_atl_{b1}_{b2}.csv', index=False)

for i in range(0,8):
    get_offset(i, min_loc_val, ROT_C[i])

(-90.78956921978838, 388.9090563626479)
(-26.409208936123278, 234.05032297215098)
(40.36048531069038, 91.38944810800395)
(13.779656426072776, 241.0624837439028)
(-33.983400360631805, 604.9009245322155)
(71.65525056191836, 467.29704157731265)
(33.21243078404612, 177.1607587515219)
(49.13851404327443, 901.7263478670337)
